In [1]:
!unzip '/content/drive/My Drive/140534_330706_bundle_archive.zip'

Streaming output truncated to the last 5000 lines.
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0031.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0032.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0033.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0034.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0035.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0036.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0037.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0038.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0039.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0040.jpg  
  inflating: 101_objectcategories/101_ObjectCategories/ceiling_fan/image_0041.jpg  
  inflating: 101_objectca

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [3]:
from torch.utils.data import Dataset, DataLoader

In [20]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [21]:
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


In [22]:
train_data = datasets.ImageFolder(root = '/content/101_ObjectCategories',transform=train_transforms)

In [23]:
len(train_data.classes)

102

In [24]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples
print(n_train_examples)
print(n_valid_examples)

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

8229
915


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
import copy
import random
import time
import torchvision.models as models

In [27]:
model = models.resnet101(pretrained=True).to(device)

for param in model.parameters():
  param.requires_grad=True
    
model.fc = nn.Sequential(
               nn.Linear(2048, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 102)).to(device)

In [28]:
BATCH_SIZE = 16
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 43,567,158 trainable parameters


In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0, dampening=0, weight_decay=0, nesterov=False)

criterion = criterion.to(device)

In [31]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [32]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 75

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 3m 20s
	Train Loss: 3.468 | Train Acc: 25.78%
	 Val. Loss: 2.707 |  Val. Acc: 35.78%
Epoch: 02 | Epoch Time: 3m 20s
	Train Loss: 2.472 | Train Acc: 46.47%
	 Val. Loss: 1.652 |  Val. Acc: 62.28%
Epoch: 03 | Epoch Time: 3m 19s
	Train Loss: 1.517 | Train Acc: 66.55%
	 Val. Loss: 0.916 |  Val. Acc: 77.08%
Epoch: 04 | Epoch Time: 3m 18s
	Train Loss: 0.950 | Train Acc: 77.64%
	 Val. Loss: 0.631 |  Val. Acc: 81.72%
Epoch: 05 | Epoch Time: 3m 19s
	Train Loss: 0.656 | Train Acc: 84.53%
	 Val. Loss: 0.462 |  Val. Acc: 86.60%
Epoch: 06 | Epoch Time: 3m 19s
	Train Loss: 0.476 | Train Acc: 88.57%
	 Val. Loss: 0.394 |  Val. Acc: 88.25%
Epoch: 07 | Epoch Time: 3m 19s
	Train Loss: 0.376 | Train Acc: 91.06%
	 Val. Loss: 0.326 |  Val. Acc: 89.76%
Epoch: 08 | Epoch Time: 3m 19s
	Train Loss: 0.294 | Train Acc: 92.73%
	 Val. Loss: 0.332 |  Val. Acc: 89.83%
Epoch: 09 | Epoch Time: 3m 21s
	Train Loss: 0.255 | Train Acc: 93.54%
	 Val. Loss: 0.303 |  Val. Acc: 90.16%
Epoch: 10 | Epoch T